# Summarization

In [ ]:
from tqdm import tqdm

In [ ]:
!pip install rouge
from rouge import Rouge 

rouge = Rouge()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Data

In [ ]:
import random
from pprint import pprint
import numpy as np
import pandas as pd
import json
from datetime import datetime as dt
from google.colab import drive

In [ ]:
drive.mount('/content/up_nlp/', force_remount=True)

Mounted at /content/up_nlp/


In [ ]:
with open("/content/up_nlp/MyDrive/up_nlp/data.json", "r") as data_file:
    data = json.load(data_file)

for news in data:
    if "$date" in news["date"].keys():
        news["date"] = dt.fromtimestamp(int(list(news["date"]["$date"].values())[0]) / 1e3)
    else:
        news["date"] = pd.NaT

len(data)

42635

In [ ]:
with_desc__indx = [
    i
    for i in range(len(data))
    if isinstance(data[i]["description"], str)
]

In [ ]:
ref = data[42024]["description"]
ref

"Federal prosecutors have charged a 32-year-old man with a federal firearm crime, alleging he sold the Colleyville, Texas, synagogue hostage-taker the gun he used during this month's 11-hour standoff."

## Extractive

### [textrank](https://github.com/summanlp/textrank)

In [ ]:
!pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54408 sha256=5e177efa3657142dde57ea9a3c622d95cc71435461135981a65858e4cd288a77
  Stored in directory: /root/.cache/pip/wheels/ed/2c/5f/a0ccc5955d44d2cea78729f4425e73f818d2629517f7af0f8b
Successfully built summa


In [ ]:
from summa.summarizer import summarize as summa_summarize

In [ ]:
%time
hyp = summa_summarize(data[42024]["content"], split=True)
hyp

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 6.68 µs


["(CNN)Federal prosecutors have charged a 32-year-old man with a federal firearm crime, alleging he sold the Colleyville, Texas, synagogue hostage-taker the gun he used during this month's 11-hour standoff.Henry Williams, who was charged Tuesday with being a felon in possession of a firearm, allegedly sold a semiautomatic pistol to Malik Faisal Akram on January 13, according to a criminal complaint.",
 'The question had been their investigation\'s "primary gap," DeSarno said last week in a news conference.Read MoreHenry Williams Authorities identified Williams by analyzing Akram\'s cell phone records, which showed the two called each other several times between January 11 and 13, according to the complaint.',
 "When authorities first approached Williams on January 16, he told federal investigators that he met with a man who had a British accent but did not remember the man's name, the complaint said.Agents interviews Williams again after he was arrested on an outstanding state warrant 

In [ ]:
summa_summarize(data[42024]["content"])

'(CNN)Federal prosecutors have charged a 32-year-old man with a federal firearm crime, alleging he sold the Colleyville, Texas, synagogue hostage-taker the gun he used during this month\'s 11-hour standoff.Henry Williams, who was charged Tuesday with being a felon in possession of a firearm, allegedly sold a semiautomatic pistol to Malik Faisal Akram on January 13, according to a criminal complaint.\nThe question had been their investigation\'s "primary gap," DeSarno said last week in a news conference.Read MoreHenry Williams Authorities identified Williams by analyzing Akram\'s cell phone records, which showed the two called each other several times between January 11 and 13, according to the complaint.\nWhen authorities first approached Williams on January 16, he told federal investigators that he met with a man who had a British accent but did not remember the man\'s name, the complaint said.Agents interviews Williams again after he was arrested on an outstanding state warrant and s

In [ ]:
rouge.get_scores(hyp[0], ref)

[{'rouge-1': {'r': 0.9615384615384616, 'p': 0.5, 'f': 0.6578947323407203},
  'rouge-2': {'r': 0.9642857142857143, 'p': 0.45, 'f': 0.6136363592975207},
  'rouge-l': {'r': 0.9615384615384616, 'p': 0.5, 'f': 0.6578947323407203}}]

In [ ]:
data[42024]["content"]

'(CNN)Federal prosecutors have charged a 32-year-old man with a federal firearm crime, alleging he sold the Colleyville, Texas, synagogue hostage-taker the gun he used during this month\'s 11-hour standoff.Henry Williams, who was charged Tuesday with being a felon in possession of a firearm, allegedly sold a semiautomatic pistol to Malik Faisal Akram on January 13, according to a criminal complaint. Two days later, Akram, a 44-year-old British national, held four people hostage at Congregation Beth Israel, the FBI has said. An FBI team killed him after hostages were released or escaped from the synagogue near Dallas, officials said. The FBI is treating the incident, which has put US Jewish communities on edge, as a hate crime and an act of terrorism.Williams had his initial court appearance Wednesday, court records show. CNN has reached out to his attorney. FBI agents and local authorities worked "around the clock" to figure out how Akram got his weapon, Matthew DeSarno, the special ag

In [ ]:
summa_scores = dict()
failed = []
for ind in tqdm(with_desc__indx):
    try:
        summa_scores[ind] = rouge.get_scores(
            summa_summarize(data[ind]["content"], split=True)[0],
            data[ind]["description"]
        )[0]
        # summa_scores[ind] = rouge.get_scores(
        #     summarize(data[ind]["content"]),
        #     data[ind]["description"]
        # )[0]
    except:
        failed.append(ind)
print(f"News failed: {failed}")
len(failed)

100%|██████████| 37899/37899 [17:31<00:00, 36.03it/s]

News failed: [4752, 4757, 4768, 4806, 4814, 4828, 4848, 4856, 4874, 4975, 4977, 4981, 4982, 4991, 5020, 5038, 5040, 5066, 5072, 5746, 5801, 5818, 5827, 5856, 5866, 5870, 5905, 5912, 5921, 5935, 5945, 5946, 5990, 6003, 6027, 6029, 6070, 6078, 6097, 6113, 6192, 6662, 6751, 7286, 7307, 7336, 7338, 7360, 7363, 7366, 7373, 7376, 7388, 7399, 7405, 7427, 7428, 7433, 7469, 7474, 7496, 7504, 7511, 7548, 7559, 7588, 7589, 7590, 7596, 7636, 7688, 7694, 7695, 7980, 8027, 8042, 8113, 8154, 8227, 8286, 8312, 8319, 8574, 8581, 8592, 8608, 8627, 8628, 8634, 8639, 8642, 8656, 8665, 8735, 8736, 8739, 8743, 8771, 8784, 8805, 8883, 8913, 8934, 8953, 8985, 9005, 9031, 9079, 9130, 9186, 9187, 9218, 9341, 9342, 9345, 9346, 9347, 9348, 9349, 9446, 9448, 9462, 9846, 9929, 9940, 9969, 9972, 10025, 10027, 10165, 10170, 10184, 10263, 10327, 10328, 10348, 10354, 10362, 10371, 10388, 10399, 10403, 10424, 10505, 10738, 10862, 10864, 10874, 10892, 10896, 10928, 11085, 11099, 11132, 11220, 11225, 11253, 11279, 11303, 

3437

In [ ]:
rouge.get_scores(
    summa_summarize(data[4752]["content"], split=True)[0],
    data[4752]["description"]
)[0]

IndexError: ignored

In [ ]:
summa_summarize(data[4752]["content"], split=True)

[]

In [ ]:
summa_score = {
    "rouge-1": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
    "rouge-2": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
    "rouge-l": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
}

In [ ]:
summa_score["rouge-1"]

{'ar': 0.0, 'ap': 0.0, 'af': 0.0, 'f1': 0.0}

In [ ]:
for score in summa_scores.values():
    summa_score["rouge-1"]["ar"] += score["rouge-1"]["r"]
    summa_score["rouge-1"]["ap"] += score["rouge-1"]["p"]
    summa_score["rouge-1"]["af"] += score["rouge-1"]["f"]

    summa_score["rouge-2"]["ar"] += score["rouge-2"]["r"]
    summa_score["rouge-2"]["ap"] += score["rouge-2"]["p"]
    summa_score["rouge-2"]["af"] += score["rouge-2"]["f"]

    summa_score["rouge-l"]["ar"] += score["rouge-l"]["r"]
    summa_score["rouge-l"]["ap"] += score["rouge-l"]["p"]
    summa_score["rouge-l"]["af"] += score["rouge-l"]["f"]

summa_score["rouge-1"]["ar"] /= len(summa_scores)
summa_score["rouge-1"]["ap"] /= len(summa_scores)
summa_score["rouge-1"]["af"] /= len(summa_scores)
summa_score["rouge-1"]["f1"] = 2 * summa_score["rouge-1"]["ar"] * summa_score["rouge-1"]["ap"] / (summa_score["rouge-1"]["ar"] + summa_score["rouge-1"]["ap"])

summa_score["rouge-2"]["ar"] /= len(summa_scores)
summa_score["rouge-2"]["ap"] /= len(summa_scores)
summa_score["rouge-2"]["af"] /= len(summa_scores)
summa_score["rouge-2"]["f1"] = 2 * summa_score["rouge-2"]["ar"] * summa_score["rouge-2"]["ap"] / (summa_score["rouge-2"]["ar"] + summa_score["rouge-2"]["ap"])

summa_score["rouge-l"]["ar"] /= len(summa_scores)
summa_score["rouge-l"]["ap"] /= len(summa_scores)
summa_score["rouge-l"]["af"] /= len(summa_scores)
summa_score["rouge-l"]["f1"] = 2 * summa_score["rouge-l"]["ar"] * summa_score["rouge-l"]["ap"] / (summa_score["rouge-l"]["ar"] + summa_score["rouge-l"]["ap"])

In [ ]:
summa_score

{'rouge-1': {'ar': 0.602127872352963,
  'ap': 0.2719255571856544,
  'af': 0.34622492011489875,
  'f1': 0.37465434412406934},
 'rouge-2': {'ar': 0.44868789282460875,
  'ap': 0.19032597545523974,
  'af': 0.2424721166800987,
  'f1': 0.26727733188851904},
 'rouge-l': {'ar': 0.5661840747674333,
  'ap': 0.25654210119705506,
  'af': 0.3262616399846347,
  'f1': 0.3530945202633673}}

### [pytextrank](https://derwen.ai/docs/ptr/start/)

In [ ]:
!pip install pytextrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.9 MB/s eta 0:00:00
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.6.1
    Uninstalling Pygments-2.6.1:
      Successfully uninstalled Pygments-2.6.1
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import spacy
pytr = spacy.load("en_core_web_sm")

import pytextrank
pytr.add_pipe("textrank", last=True)

from math import sqrt
from operator import itemgetter

In [ ]:
def pytr_summarize(text: str, limit_phrases=5, limit_sentences=None, ratio=0.2):
    doc = pytr(text)
    sents = list(doc.sents)
    sent_bounds = [[s.start, s.end, set([])] for s in sents]
    if limit_sentences is None:
        limit_sentences = int(len(sents) * ratio)

    # top-ranked phrases
    phrase_id = 0
    unit_vector = []
    for p in doc._.phrases:
        unit_vector.append(p.rank)
        for chunk in p.chunks:
            for sent_start, sent_end, sent_vector in sent_bounds:
                if chunk.start >= sent_start and chunk.end <= sent_end:
                    sent_vector.add(phrase_id)
                    break

        phrase_id += 1
        if phrase_id == limit_phrases:
            break

    sum_ranks = sum(unit_vector)
    unit_vector = [rank/sum_ranks for rank in unit_vector]

    # minimal distance sentences
    sent_rank = {}
    sent_id = 0
    for sent_start, sent_end, sent_vector in sent_bounds:
        sum_sq = 0.0
        for phrase_id in range(len(unit_vector)):
            if phrase_id not in sent_vector:
                sum_sq += unit_vector[phrase_id]**2.0
        sent_rank[sent_id] = sqrt(sum_sq)
        sent_id += 1
    sent_rank = sorted(sent_rank.items(), key=itemgetter(1)) 

    # соблюдать очерёдность предложений в тексте
    return " ".join([sents[sent_id].text for sent_id, rank in sent_rank[:limit_sentences]])

In [ ]:
ref 

"Federal prosecutors have charged a 32-year-old man with a federal firearm crime, alleging he sold the Colleyville, Texas, synagogue hostage-taker the gun he used during this month's 11-hour standoff."

In [ ]:
hyp = pytr_summarize(data[42024]["content"], limit_phrases=20)
hyp

'Akram arrived in the United States in late December via New York\'s John F. Kennedy International Airport, a US law enforcement source familiar with the investigation told CNN.Akram was known to UK security services and was the subject of a brief investigation in 2020, a UK official told CNN. Last week, two men were arrested in Birmingham and Manchester in connection with the standoff investigation and were held for questioning, Greater Manchester Police said at the time without elaborating. Jewish communities across the US are on heightened alert after the Texas standoff: \'Is our community under attack again?\'US investigators believe Akram was motivated in part by a desire to see the release of extremist Aafia Siddiqui, who is serving an 86-year federal prison sentence in Fort Worth, Texas, they\'ve said. UK counterterrorism investigators have been helping US authorities look into the incident. And while the majority of anti-Semitic incidents involve harassment and vandalism, assau

In [ ]:
rouge.get_scores(hyp, ref)

[{'rouge-1': {'r': 0.3076923076923077,
   'p': 0.05194805194805195,
   'f': 0.08888888641728403},
  'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
  'rouge-l': {'r': 0.2692307692307692,
   'p': 0.045454545454545456,
   'f': 0.07777777530617291}}]

In [ ]:
pytr_scores = dict()
failed = []
for ind in tqdm(with_desc__indx):
    try:
        pytr_scores[ind] = rouge.get_scores(
            pytr_summarize(data[ind]["content"]),
            data[ind]["description"]
        )[0]
    except:
        failed.append(ind)
print(f"News failed: {failed}")
len(failed)

  0%|          | 143/37899 [00:37<1:46:55,  5.89it/s]

In [ ]:
pytr_score = {
    "rouge-1": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
    "rouge-2": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
    "rouge-l": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
}

In [ ]:
for score in pytr_scores.values():
    pytr_score["rouge-1"]["ar"] += score["rouge-1"]["r"]
    pytr_score["rouge-1"]["ap"] += score["rouge-1"]["p"]
    pytr_score["rouge-1"]["af"] += score["rouge-1"]["f"]

    pytr_score["rouge-2"]["ar"] += score["rouge-2"]["r"]
    pytr_score["rouge-2"]["ap"] += score["rouge-2"]["p"]
    pytr_score["rouge-2"]["af"] += score["rouge-2"]["f"]

    pytr_score["rouge-l"]["ar"] += score["rouge-l"]["r"]
    pytr_score["rouge-l"]["ap"] += score["rouge-l"]["p"]
    pytr_score["rouge-l"]["af"] += score["rouge-l"]["f"]

pytr_score["rouge-1"]["ar"] /= len(pytr_scores)
pytr_score["rouge-1"]["ap"] /= len(pytr_scores)
pytr_score["rouge-1"]["af"] /= len(pytr_scores)
pytr_score["rouge-1"]["f1"] = 2 * pytr_score["rouge-1"]["ar"] * pytr_score["rouge-1"]["ap"] / (pytr_score["rouge-1"]["ar"] + pytr_score["rouge-1"]["ap"])

pytr_score["rouge-2"]["ar"] /= len(pytr_scores)
pytr_score["rouge-2"]["ap"] /= len(pytr_scores)
pytr_score["rouge-2"]["af"] /= len(pytr_scores)
pytr_score["rouge-2"]["f1"] = 2 * pytr_score["rouge-2"]["ar"] * pytr_score["rouge-2"]["ap"] / (pytr_score["rouge-2"]["ar"] + pytr_score["rouge-2"]["ap"])

pytr_score["rouge-l"]["ar"] /= len(pytr_scores)
pytr_score["rouge-l"]["ap"] /= len(pytr_scores)
pytr_score["rouge-l"]["af"] /= len(pytr_scores)
pytr_score["rouge-l"]["f1"] = 2 * pytr_score["rouge-l"]["ar"] * pytr_score["rouge-l"]["ap"] / (pytr_score["rouge-l"]["ar"] + pytr_score["rouge-l"]["ap"])

In [ ]:
pytr_score

{'rouge-1': {'ar': 0.33346773442920646,
  'ap': 0.24696868111114315,
  'af': 0.26776070428136894,
  'f1': 0.28377298308698895},
 'rouge-2': {'ar': 0.20088304340564977,
  'ap': 0.13133254123262753,
  'af': 0.14870959516480037,
  'f1': 0.158827471081671},
 'rouge-l': {'ar': 0.29529481047866896,
  'ap': 0.216488324093069,
  'af': 0.23574288027003876,
  'f1': 0.24982409272788}}

## Abstractive

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 4.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00


In [3]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### [PEGASUS$_{LARGE}$](https://huggingface.co/google/pegasus-large)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

pegasus_tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
pegasus_model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-large").to(device)

In [ ]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

pegasus_tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
pegasus_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum").to(device)

In [ ]:
def pegasus_summarize(news: str):
    pegasus_inputs = pegasus_tokenizer(news, truncation=True, return_tensors="pt").to(device)
    pegasus_summary = pegasus_model.generate(**pegasus_inputs)
    return pegasus_tokenizer.batch_decode(pegasus_summary, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
%time hyp = pegasus_summarize(data[42024]["content"])
hyp

CPU times: user 5.7 s, sys: 0 ns, total: 5.7 s
Wall time: 8 s


'(CNN)Federal prosecutors have charged a 32-year-old man with a federal firearm crime, alleging he sold the Colleyville, Texas, synagogue hostage-taker the gun he used during this month\'s 11-hour standoff. The question had been their investigation\'s "primary gap," DeSarno said last week in a news conference.Read MoreHenry Williams Authorities identified Williams by analyzing Akram\'s cell phone records, which showed the two called each other several times between January 11 and 13, according to the complaint.'

In [ ]:
ref

"Federal prosecutors have charged a 32-year-old man with a federal firearm crime, alleging he sold the Colleyville, Texas, synagogue hostage-taker the gun he used during this month's 11-hour standoff."

In [ ]:
rouge.get_scores(hyp, ref)

[{'rouge-1': {'r': 0.9615384615384616,
   'p': 0.36764705882352944,
   'f': 0.5319148896152105},
  'rouge-2': {'r': 0.9642857142857143,
   'p': 0.36486486486486486,
   'f': 0.5294117607227989},
  'rouge-l': {'r': 0.9615384615384616,
   'p': 0.36764705882352944,
   'f': 0.5319148896152105}}]

In [ ]:
pegasus_scores = dict()
failed = []
for ind in tqdm(with_desc__indx):
    try:
        pegasus_scores[ind] = rouge.get_scores(
            pegasus_summarize(data[ind]["content"]),
            data[ind]["description"]
        )[0]
    except:
        failed.append(ind)
print(f"News failed: {failed}")
len(failed)

  0%|          | 87/37899 [01:18<10:14:17,  1.03it/s]

In [ ]:
pegasus_score = {
    "rouge-1": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
    "rouge-2": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
    "rouge-l": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
}

In [ ]:
for score in pegasus_scores.values():
    pegasus_score["rouge-1"]["ar"] += score["rouge-1"]["r"]
    pegasus_score["rouge-1"]["ap"] += score["rouge-1"]["p"]
    pegasus_score["rouge-1"]["af"] += score["rouge-1"]["f"]

    pegasus_score["rouge-2"]["ar"] += score["rouge-2"]["r"]
    pegasus_score["rouge-2"]["ap"] += score["rouge-2"]["p"]
    pegasus_score["rouge-2"]["af"] += score["rouge-2"]["f"]

    pegasus_score["rouge-l"]["ar"] += score["rouge-l"]["r"]
    pegasus_score["rouge-l"]["ap"] += score["rouge-l"]["p"]
    pegasus_score["rouge-l"]["af"] += score["rouge-l"]["f"]

pegasus_score["rouge-1"]["ar"] /= len(pegasus_scores)
pegasus_score["rouge-1"]["ap"] /= len(pegasus_scores)
pegasus_score["rouge-1"]["af"] /= len(pegasus_scores)
pegasus_score["rouge-1"]["f1"] = 2 * pegasus_score["rouge-1"]["ar"] * pegasus_score["rouge-1"]["ap"] / (pegasus_score["rouge-1"]["ar"] + pegasus_score["rouge-1"]["ap"])

pegasus_score["rouge-2"]["ar"] /= len(pegasus_scores)
pegasus_score["rouge-2"]["ap"] /= len(pegasus_scores)
pegasus_score["rouge-2"]["af"] /= len(pegasus_scores)
pegasus_score["rouge-2"]["f1"] = 2 * pegasus_score["rouge-2"]["ar"] * pegasus_score["rouge-2"]["ap"] / (pegasus_score["rouge-2"]["ar"] + pegasus_score["rouge-2"]["ap"])

pegasus_score["rouge-l"]["ar"] /= len(pegasus_scores)
pegasus_score["rouge-l"]["ap"] /= len(pegasus_scores)
pegasus_score["rouge-l"]["af"] /= len(pegasus_scores)
pegasus_score["rouge-l"]["f1"] = 2 * pegasus_score["rouge-l"]["ar"] * pegasus_score["rouge-l"]["ap"] / (pegasus_score["rouge-l"]["ar"] + pegasus_score["rouge-l"]["ap"])

In [ ]:
pegasus_score

### [T5 finetuned on news](https://huggingface.co/mrm8488/t5-base-finetuned-summarize-news)

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

t5_tokenizer = AutoTokenizer.from_pretrained("t5-large")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-large").to(device)

def t5_summarize(text, max_length=150):
    input_ids = t5_tokenizer("summarize: " + text, truncation=True, return_tensors="pt", add_special_tokens=True).to(device)
    generated_ids = t5_model.generate(input_ids=input_ids, max_length=max_length, early_stopping=True)
    preds = t5_tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return preds[:preds.rfind(".")+1]

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

t5_tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
t5_model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-summarize-news").to(device)

def t5_summarize(text, max_length=150):
    input_ids = t5_tokenizer.encode(text, return_tensors="pt", add_special_tokens=True).to(device)
    generated_ids = t5_model.generate(input_ids=input_ids, max_length=max_length, early_stopping=True)
    preds = t5_tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return preds[:preds.rfind(".")+1]

In [ ]:
%time hyp = t5_summarize(data[42024]["content"])
hyp

Token indices sequence length is longer than the specified maximum sequence length for this model (1031 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 2.83 s, sys: 16.5 ms, total: 2.85 s
Wall time: 3.72 s


'FBI has charged a 32-year-old man with a federal firearm crime for allegedly selling a semiautomatic pistol to a Texas synagogue hostage taker. The accused, Malik Faisal Akram, allegedly sold the gun he used during the Texas standoff to the hostage-taker. The FBI is treating the incident, which took place in December, as a hate crime and an act of terrorism. The FBI is treating the case as a hate crime.'

In [ ]:
ref

"Federal prosecutors have charged a 32-year-old man with a federal firearm crime, alleging he sold the Colleyville, Texas, synagogue hostage-taker the gun he used during this month's 11-hour standoff."

In [ ]:
rouge.get_scores(hyp, ref)

[{'rouge-1': {'r': 0.6153846153846154,
   'p': 0.32653061224489793,
   'f': 0.4266666621368889},
  'rouge-2': {'r': 0.42857142857142855,
   'p': 0.19047619047619047,
   'f': 0.26373625947590873},
  'rouge-l': {'r': 0.5769230769230769,
   'p': 0.30612244897959184,
   'f': 0.39999999547022225}}]

In [ ]:
t5_scores = dict()
failed = []
for ind in tqdm(with_desc__indx):
    try:
        t5_scores[ind] = rouge.get_scores(
            t5_summarize(data[ind]["content"]),
            data[ind]["description"]
        )[0]
    except:
        failed.append(ind)
print(f"News failed: {failed}")
len(failed)

In [ ]:
t5_score = {
    "rouge-1": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
    "rouge-2": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
    "rouge-l": {
        "ar": 0.0, "ap": 0.0, "af": 0.0, "f1": 0.0},
}

In [ ]:
for score in t5_scores.values():
    t5_score["rouge-1"]["ar"] += score["rouge-1"]["r"]
    t5_score["rouge-1"]["ap"] += score["rouge-1"]["p"]
    t5_score["rouge-1"]["af"] += score["rouge-1"]["f"]

    t5_score["rouge-2"]["ar"] += score["rouge-2"]["r"]
    t5_score["rouge-2"]["ap"] += score["rouge-2"]["p"]
    t5_score["rouge-2"]["af"] += score["rouge-2"]["f"]

    t5_score["rouge-l"]["ar"] += score["rouge-l"]["r"]
    t5_score["rouge-l"]["ap"] += score["rouge-l"]["p"]
    t5_score["rouge-l"]["af"] += score["rouge-l"]["f"]

t5_score["rouge-1"]["ar"] /= len(t5_scores)
t5_score["rouge-1"]["ap"] /= len(t5_scores)
t5_score["rouge-1"]["af"] /= len(t5_scores)
t5_score["rouge-1"]["f1"] = 2 * t5_score["rouge-1"]["ar"] * t5_score["rouge-1"]["ap"] / (t5_score["rouge-1"]["ar"] + t5_score["rouge-1"]["ap"])

t5_score["rouge-2"]["ar"] /= len(t5_scores)
t5_score["rouge-2"]["ap"] /= len(t5_scores)
t5_score["rouge-2"]["af"] /= len(t5_scores)
t5_score["rouge-2"]["f1"] = 2 * t5_score["rouge-2"]["ar"] * t5_score["rouge-2"]["ap"] / (t5_score["rouge-2"]["ar"] + t5_score["rouge-2"]["ap"])

t5_score["rouge-l"]["ar"] /= len(t5_scores)
t5_score["rouge-l"]["ap"] /= len(t5_scores)
t5_score["rouge-l"]["af"] /= len(t5_scores)
t5_score["rouge-l"]["f1"] = 2 * t5_score["rouge-l"]["ar"] * t5_score["rouge-l"]["ap"] / (t5_score["rouge-l"]["ar"] + t5_score["rouge-l"]["ap"])

In [ ]:
t5_score

## DEMO

In [ ]:
news = data[random.choice(with_desc__indx)]
news

{'index': 39475,
 'source': 'CNN',
 'date': datetime.datetime(2021, 3, 12, 0, 0),
 'title': 'Sarah Everard: Police identify body of woman who went missing in London - CNN',
 'category': 'news',
 'description': 'British police confirmed Friday that they had identified the body of 33-year-old Sarah Everard, whose disappearance while walking home in London sparked an outpouring across social media from women sharing their own experiences of sexual assault and harassment.',
 'content': 'London (CNN)British police confirmed Friday that they had identified the body of 33-year-old Sarah Everard, whose disappearance while walking home in London sparked an outpouring across social media from women sharing their own experiences of sexual assault and harassment.The UK\'s Crown Prosecution Service authorized police to charge a serving Metropolitan Police officer with Everard\'s murder and kidnapping on Friday, according to a statement from Rosemary Ainslie, the CPS\' head of special crime. The sta

In [ ]:
summaries = {
    "reference": news["description"],
    "summa": summa_summarize(news["content"]),
    "pytextrank": pytr_summarize(news["content"]),
    "pegasus": pegasus_summarize(news["content"]),
    "t5": t5_summarize(news["content"])
}
summaries

{'reference': 'British police confirmed Friday that they had identified the body of 33-year-old Sarah Everard, whose disappearance while walking home in London sparked an outpouring across social media from women sharing their own experiences of sexual assault and harassment.',
 'summa': 'London (CNN)British police confirmed Friday that they had identified the body of 33-year-old Sarah Everard, whose disappearance while walking home in London sparked an outpouring across social media from women sharing their own experiences of sexual assault and harassment.The UK\'s Crown Prosecution Service authorized police to charge a serving Metropolitan Police officer with Everard\'s murder and kidnapping on Friday, according to a statement from Rosemary Ainslie, the CPS\' head of special crime.\nA police officer whose "primary role was uniformed patrol duties of diplomatic premises" was arrested in Kent on Tuesday evening.Sarah Everard case prompts outpouring from women sharing stories of abuse a

In [ ]:
scores = {
    "summa": rouge.get_scores(summaries["summa"], summaries["reference"]),
    "pytextrank": rouge.get_scores(summaries["pytextrank"], summaries["reference"]),
    "pegasus": rouge.get_scores(summaries["pegasus"], summaries["reference"]),
    "t5": rouge.get_scores(summaries["t5"], summaries["reference"])
}
scores

{'summa': [{'rouge-1': {'r': 0.972972972972973,
    'p': 0.33962264150943394,
    'f': 0.5034964996606192},
   'rouge-2': {'r': 0.972972972972973,
    'p': 0.2571428571428571,
    'f': 0.4067796577101088},
   'rouge-l': {'r': 0.972972972972973,
    'p': 0.33962264150943394,
    'f': 0.5034964996606192}}],
 'pytextrank': [{'rouge-1': {'r': 0.972972972972973,
    'p': 0.41379310344827586,
    'f': 0.5806451571032779},
   'rouge-2': {'r': 0.972972972972973,
    'p': 0.3333333333333333,
    'f': 0.49655172033674205},
   'rouge-l': {'r': 0.972972972972973,
    'p': 0.41379310344827586,
    'f': 0.5806451571032779}}],
 'pegasus': [{'rouge-1': {'r': 0.972972972972973,
    'p': 0.5625,
    'f': 0.7128712824860309},
   'rouge-2': {'r': 0.972972972972973, 'p': 0.5, 'f': 0.6605504542311254},
   'rouge-l': {'r': 0.972972972972973, 'p': 0.5625, 'f': 0.7128712824860309}}],
 't5': [{'rouge-1': {'r': 0.2702702702702703,
    'p': 0.20408163265306123,
    'f': 0.23255813463223374},
   'rouge-2': {'r': 0